- 准备数据集，处理 gorilla 的 instruction + code example
    - Instruction 任务说明
    - Function，接受端到端任务
    - Test function
    - Test dataset

In [1]:
import json
import pprint

def load_jsonl_data(path):
    data = []
    with open(path) as f:
        for l in f:
            d = json.loads(l)
            data.append(d)
            
    return data

hf_api_data = load_jsonl_data("gorilla/data/api/huggingface_api.jsonl")
len(hf_api_data), hf_api_data[0]

(936,
 {'domain': 'Natural Language Processing Feature Extraction',
  'framework': 'Hugging Face Transformers',
  'functionality': 'Feature Extraction',
  'api_name': 'YituTech/conv-bert-base',
  'api_call': "AutoModel.from_pretrained('YituTech/conv-bert-base')",
  'api_arguments': 'N/A',
  'python_environment_requirements': 'transformers',
  'example_code': 'N/A',
  'performance': {'dataset': 'N/A', 'accuracy': 'N/A'},
  'description': 'A pre-trained ConvBERT model for feature extraction provided by YituTech, based on the Hugging Face Transformers library.'})

In [2]:
from collections import Counter

domain_counter_dict = Counter()

for d in hf_api_data:
    domain_counter_dict[d['domain']] += 1
        
pprint.pp(domain_counter_dict.most_common())

[('Natural Language Processing Text2Text Generation', 41),
 ('Natural Language Processing Text Generation', 39),
 ('Natural Language Processing Sentence Similarity', 33),
 ('Computer Vision Image Classification', 33),
 ('Natural Language Processing Token Classification', 33),
 ('Natural Language Processing Zero-Shot Classification', 33),
 ('Natural Language Processing Text Classification', 32),
 ('Audio Automatic Speech Recognition', 31),
 ('Natural Language Processing Table Question Answering', 31),
 ('Computer Vision Video Classification', 30),
 ('Multimodal Text-to-Image', 30),
 ('Multimodal Image-to-Text', 30),
 ('Computer Vision Object Detection', 30),
 ('Computer Vision Image Segmentation', 30),
 ('Natural Language Processing Fill-Mask', 30),
 ('Natural Language Processing Question Answering', 29),
 ('Multimodal Document Question Answer', 29),
 ('Computer Vision Depth Estimation', 29),
 ('Computer Vision Unconditional Image Generation', 29),
 ('Audio Text-to-Speech', 29),
 ('Audi

In [3]:
hf_train_data = load_jsonl_data("gorilla/data/apibench/huggingface_train.json")
len(hf_train_data), hf_train_data[0]

(8191,
 {'code': "###Instruction: Write an API implementation that takes customer reviews as input and extracts features to analyze customer sentiment.\n###Output: <<<domain>>>: Natural Language Processing Feature Extraction\n<<<api_call>>>: AutoModel.from_pretrained('YituTech/conv-bert-base')\n<<<api_provider>>>: Hugging Face Transformers\n<<<explanation>>>: 1. We import the necessary classes from the transformers package. This includes AutoTokenizer and AutoModel for tokenizing and processing customer review text.\n2. We use the from_pretrained method of the AutoModel class to load the pre-trained model 'YituTech/conv-bert-base'. This model is based on ConvBERT and is suitable for feature extraction in text data.\n3. We load the customer review text, tokenize it, and use the model to extract features from the review. These features can then be used to analyze customer sentiment.\n<<<code>>>: from transformers import AutoTokenizer, AutoModel\ntokenizer = AutoTokenizer.from_pretrained(

In [4]:
hf_eval_data = load_jsonl_data("gorilla/data/apibench/huggingface_eval.json")
len(hf_eval_data), hf_eval_data[1]

(911,
 {'code': '###Instruction: The user is interested in a tool to find relationships between medical terms.\n###Output: <<<domain>>>: Multimodal Feature Extraction\n<<<api_call>>>: AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n<<<api_provider>>>: Hugging Face Transformers\n<<<explanation>>>: 1. We import the necessary classes from the transformers package provided by Hugging Face.\n2. We then call the "AutoModel.from_pretrained" method with the argument \'GanjinZero/UMLSBert_ENG\' to load this pretrained model.\n3. This model, which is particularly suitable for finding relationships between medical terms, can be used to convert medical terms into embeddings (dense vectors).\n4. These embeddings can then be compared to find similarities and relationships between various medical terms.\n<<<code>>>: from transformers import AutoTokenizer, AutoModel\ntokenizer = AutoTokenizer.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\nmodel = AutoModel.from_pretrained(\'GanjinZero/UMLSBert_

# 1. Instruction

In [5]:
# instruction: apibench - {lib}_train.json - code - instruction

import re

def get_code_parts_from_apibench_data(data):
    text = data['code']
    instruction, _ = text.split("\n###Output")
    
    # Extracting domain, api_call, api_provider, and code using regular expressions
    domain_pattern = r'<<<domain>>>: (.+?)\n'
    api_call_pattern = r'<<<api_call>>>: (.+?)\n'
    api_provider_pattern = r'<<<api_provider>>>: (.+?)\n'
    code_pattern = r'<<<code>>>: (.+)'

    domain = re.search(domain_pattern, text).group(1)
    api_call = re.search(api_call_pattern, text).group(1)
    api_provider = re.search(api_provider_pattern, text).group(1)
    code = re.search(code_pattern, text, re.DOTALL).group(1).strip()

    return {
        'instruction': instruction, 
        'domain': domain, 
        'api_call': api_call, 
        'api_provider': api_provider, 
        'code': code
    }

d = hf_eval_data[0]
code_parts = get_code_parts_from_apibench_data(d)
code_parts, code_parts['instruction']

({'instruction': '###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.',
  'domain': 'Natural Language Processing Sentence Similarity',
  'api_call': "AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')",
  'api_provider': 'Hugging Face Transformers',
  'code': "from transformers import AutoTokenizer, AutoModel\ntokenizer = AutoTokenizer.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\nmodel = AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')"},
 '###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.')

In [6]:
for d in hf_eval_data:
    code_parts = get_code_parts_from_apibench_data(d)
    print(code_parts['instruction'])
    pprint.pp(d)
    break

###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.
{'code': '###Instruction: Design a feature for a social media website to '
         'recommend articles to users based on how similar the articles are to '
         'their previously liked articles.\n'
         '###Output: <<<domain>>>: Natural Language Processing Sentence '
         'Similarity\n'
         '<<<api_call>>>: '
         "AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\n"
         '<<<api_provider>>>: Hugging Face Transformers\n'
         '<<<explanation>>>:1. We first import the necessary classes and '
         'modules from the transformers package. This includes AutoTokenizer '
         'and AutoModel for loading the pre-trained models from Hugging Face.\n'
         '2. We use the AutoModel.from_pretrained() method to load the '
         "'princeton-nlp/unsup-simcse-roberta-base' model,

# 2. Function / Test Function
- code part -> gpt -> function
- dataset 问题，先通过 prompt 解决一部分，需要对应到 huggingface dataset 名称才能对应
- prompt:
    generate following code based on above infomation:
    1. function with：
    - detailed comments
    - function description
    2. test function with：
    - test dataset
    - using assert in test function
    - do not compare number strictly
    - if dataset is provided in performance - dataset, load the dataset, then select several sample from the dataset, otherwise, using online source, do not leave blank


In [7]:
! pip install langchain -U

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [49]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
    create_openai_fn_runnable,
    create_structured_output_runnable,
)
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

def get_chat_model():
    BASE_URL = "https://autoagents-ca-east.openai.azure.com/"
    API_KEY = "2864ce19a46540b2a0943df607ca6225"
    model = AzureChatOpenAI(
        temperature=0.0,
        openai_api_base=BASE_URL,
        openai_api_version="2023-08-01-preview",
        deployment_name="gpt-4-32k",
        openai_api_key=API_KEY,
        openai_api_type="azure",
    )
    return model

# def get_chat_model():
#     BASE_URL = "https://autoagents-global.openai.azure.com"
#     API_KEY = "6c1c61bd992146a1bbcde4a80fef51ba"
#     model = AzureChatOpenAI(
#         temperature=0.0,
#         openai_api_base=BASE_URL,
#         openai_api_version="2023-08-01-preview",
#         deployment_name="gpt-35-turbo-16k",
#         openai_api_key=API_KEY,
#         openai_api_type="azure",
#     )
#     return model


######################################################################
from langchain.pydantic_v1 import BaseModel, Field

class CodeResp(BaseModel):
    """
    generate function_code and test_function_code based on input.
    
    function comment can be descripted as following
    
    Certain aspects of a function should be documented in special sections, listed below. Each section begins with a heading line, which ends with a colon. All sections other than the heading should maintain a hanging indent of two or four spaces (be consistent within a file). These sections can be omitted in cases where the function’s name and signature are informative enough that it can be aptly described using a one-line docstring.

    Args:
        List each parameter by name. A description should follow the name, and be separated by a colon followed by either a space or newline. If the description is too long to fit on a single 80-character line, use a hanging indent of 2 or 4 spaces more than the parameter name (be consistent with the rest of the docstrings in the file). The description should include required type(s) if the code does not contain a corresponding type annotation. If a function accepts *foo (variable length argument lists) and/or **bar (arbitrary keyword arguments), they should be listed as *foo and **bar.
    Returns: (or Yields: for generators)
        Describe the semantics of the return value, including any type information that the type annotation does not provide. If the function only returns None, this section is not required. It may also be omitted if the docstring starts with Returns or Yields (e.g. \"\"\"Returns row from Bigtable as a tuple of strings.\"\"\") and the opening sentence is sufficient to describe the return value. Do not imitate older ‘NumPy style’ (example), which frequently documented a tuple return value as if it were multiple return values with individual names (never mentioning the tuple). Instead, describe such a return value as: “Returns: A tuple (mat_a, mat_b), where mat_a is …, and …”. The auxiliary names in the docstring need not necessarily correspond to any internal names used in the function body (as those are not part of the API).
    Raises:
        List all exceptions that are relevant to the interface followed by a description. Use a similar exception name + colon + space or newline and hanging indent style as described in Args:. You should not document exceptions that get raised if the API specified in the docstring is violated (because this would paradoxically make behavior under violation of the API part of the API).
    """
    function_name: str = Field(..., description="function name")
    function_import: str = Field(..., description="import nessary lib before function code")
    function_code: str = Field(..., description="standalone function with:\n- detailed comments\n- function description")
    test_function_code: str = Field(..., description="standalone test function")
    call_test_function_code: str = Field(..., description="test function call in the end of test function file")
    function_import_fixed: str = Field(..., description="import nessary lib before function code, fixed by checking the function, e.g. 'import torch' or 'import numpy as np' if needed")
    requirements_file: str =  Field(..., description="dependency packages needed to install, usage: pip install -r requirements.txt, do not specific the version")
    
######################################################################
from langchain.chains.openai_functions import (
    convert_to_openai_function,
    get_openai_output_parser,
)
    
def get_function_from_data(data, err):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a world class algorithm for recording entities."),
            (
                "human",
                """
basic info:
{input}
err to be fixed:
{err}
generate following code based on above information:
1. function with:
    - detailed comments
    - function description
    - not using " in file
2. test function with:
    - 3-5 test cases
        - for multi-media task: try to load official dataset, or using online resource url
        - e.g. https://placekitten.com/200/300
    - using assert in test function
    - do not compare number strictly
    - return 'All Tests Passed' in final if every assertion passed
    - do not use fake file url in test case
    - not using " in file
3. requirements_file with no specific package version
""",
            ),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )
    
    llm = get_chat_model()
    runnable = create_structured_output_runnable(CodeResp, llm, prompt)
    
    resp = runnable.invoke({"input": str(data), "err": str(err)})
    return resp

import glob

def get_v2_err(base_dir, idx):
    formatted_number = str(idx).zfill(5)
    matching_files = glob.glob(f"{base_dir}/f{formatted_number}_*.err")
    for file_path in matching_files:
        lines = []
        with open(file_path) as f:
            for idx, l in enumerate(f):
                if 'Downloading' in l:
                    continue
                lines.append(l)
                
            return "".join(lines[-50:])
    
    return ""

In [50]:
idx = 0
print(hf_eval_data[idx])

err = get_v2_err("output/hf-eval-data-v2", idx + 1)
print(err)

resp = get_function_from_data(hf_eval_data[idx], err)
resp

{'code': "###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.\n###Output: <<<domain>>>: Natural Language Processing Sentence Similarity\n<<<api_call>>>: AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\n<<<api_provider>>>: Hugging Face Transformers\n<<<explanation>>>:1. We first import the necessary classes and modules from the transformers package. This includes AutoTokenizer and AutoModel for loading the pre-trained models from Hugging Face.\n2. We use the AutoModel.from_pretrained() method to load the 'princeton-nlp/unsup-simcse-roberta-base' model, which is specially designed for calculating sentence similarity.\n3. To build the recommendation feature, we process the text of previously liked articles and compute sentence embeddings. For each new article, we compute its sentence embedding and compare it to the embeddings of previously liked articles.\n

ValidationError: 1 validation error for _OutputFormatter
output
  field required (type=value_error.missing)

In [40]:
print(hf_eval_data[idx])
print("----------------------")
print(resp.requirements_file)
print("----------------------")
print(resp.function_import)
print("----------------------")
print(resp.function_import_fixed)
print("----------------------")
print(resp.function_code)
print("----------------------")
print(resp.test_function_code)
print("----------------------")
print(resp.call_test_function_code)

{'code': "###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.\n###Output: <<<domain>>>: Natural Language Processing Sentence Similarity\n<<<api_call>>>: AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\n<<<api_provider>>>: Hugging Face Transformers\n<<<explanation>>>:1. We first import the necessary classes and modules from the transformers package. This includes AutoTokenizer and AutoModel for loading the pre-trained models from Hugging Face.\n2. We use the AutoModel.from_pretrained() method to load the 'princeton-nlp/unsup-simcse-roberta-base' model, which is specially designed for calculating sentence similarity.\n3. To build the recommendation feature, we process the text of previously liked articles and compute sentence embeddings. For each new article, we compute its sentence embedding and compare it to the embeddings of previously liked articles.\n

In [51]:
import traceback

err_dir = "output/hf-eval-data-v2"
output_dir = "output/hf-eval-data-v3"

for idx, d in enumerate(hf_eval_data):
    print(idx + 1, end="...")
    formatted_number = str(idx + 1).zfill(5)
    
    # 跳过已生成的
    matching_files = glob.glob(f"{output_dir}/f{formatted_number}_*")
    if matching_files:
        print("skip", end="...")
        continue
    
    max_retries = 3
    retry_count = 0

    while retry_count < max_retries:
        try:
            err = get_v2_err(err_dir, idx + 1)
            resp = get_function_from_data(d, err)

            # 写入 prompt
            with open(f"{output_dir}/f{formatted_number}_{resp.function_name}.prompt", 'w') as f:
                f.write(str(d) + "\n\n")
                f.write(str(err))

            # 写入 requirements
            with open(f"{output_dir}/f{formatted_number}_{resp.function_name}.txt", 'w') as f:
                f.write(resp.requirements_file)

            # 写入 python
            with open(f"{output_dir}/f{formatted_number}_{resp.function_name}.py", 'w') as f:
                f.write("# function_import --------------------\n\n")
                f.write(resp.function_import_fixed)

                f.write("\n\n# function_code --------------------\n\n")
                f.write(resp.function_code)

                f.write("\n\n# test_function_code --------------------\n\n")
                f.write(resp.test_function_code)

                f.write("\n\n# call_test_function_code --------------------\n\n")
                f.write(resp.call_test_function_code)

            break  # Break out of the loop if the operation is successful
        except Exception:
            retry_count += 1
            if retry_count < max_retries:
                print(f"Retrying... (Attempt {retry_count}/{max_retries})", end="...")
            else:
                print("Max retries reached. Exiting.", end="...")
                traceback.print_exc()


1...skip...2...skip...3...skip...4...skip...5...skip...6...skip...7...skip...8...Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...Max retries reached. Exiting....9...

Traceback (most recent call last):
  File "/tmp/ipykernel_4449/1340527589.py", line 22, in <cell line: 6>
    resp = get_function_from_data(d, err)
  File "/tmp/ipykernel_4449/1390181831.py", line 106, in get_function_from_data
    resp = runnable.invoke({"input": str(data), "err": str(err)})
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 1213, in invoke
    input = step.invoke(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/output_parser.py", line 85, in invoke
    return self._call_with_config(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 715, in _call_with_config
    output = call_func_with_variable_args(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/config.py", line 308, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ignore[call-arg]
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/out

Retrying... (Attempt 1/3)...10...Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...11...Retrying... (Attempt 1/3)...12...Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...Max retries reached. Exiting....13...

Traceback (most recent call last):
  File "/tmp/ipykernel_4449/1340527589.py", line 22, in <cell line: 6>
    resp = get_function_from_data(d, err)
  File "/tmp/ipykernel_4449/1390181831.py", line 106, in get_function_from_data
    resp = runnable.invoke({"input": str(data), "err": str(err)})
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 1213, in invoke
    input = step.invoke(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/output_parser.py", line 85, in invoke
    return self._call_with_config(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 715, in _call_with_config
    output = call_func_with_variable_args(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/config.py", line 308, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ignore[call-arg]
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/out

Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...Max retries reached. Exiting....14...

...

Retrying... (Attempt 1/3)...40...Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...Max retries reached. Exiting....41...

Traceback (most recent call last):
  File "/tmp/ipykernel_4449/1340527589.py", line 22, in <cell line: 6>
    resp = get_function_from_data(d, err)
  File "/tmp/ipykernel_4449/1390181831.py", line 106, in get_function_from_data
    resp = runnable.invoke({"input": str(data), "err": str(err)})
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 1213, in invoke
    input = step.invoke(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/output_parser.py", line 85, in invoke
    return self._call_with_config(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 715, in _call_with_config
    output = call_func_with_variable_args(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/config.py", line 308, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ignore[call-arg]
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/out

42...Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...43...Retrying... (Attempt 1/3)...Retrying... (Attempt 2/3)...Max retries reached. Exiting....44...

Traceback (most recent call last):
  File "/tmp/ipykernel_4449/1340527589.py", line 22, in <cell line: 6>
    resp = get_function_from_data(d, err)
  File "/tmp/ipykernel_4449/1390181831.py", line 106, in get_function_from_data
    resp = runnable.invoke({"input": str(data), "err": str(err)})
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 1213, in invoke
    input = step.invoke(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/output_parser.py", line 85, in invoke
    return self._call_with_config(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/base.py", line 715, in _call_with_config
    output = call_func_with_variable_args(
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/runnable/config.py", line 308, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ignore[call-arg]
  File "/root/miniconda3/lib/python3.8/site-packages/langchain/schema/out

KeyboardInterrupt: 

# 3. Dataset
- search from {lib}